In [1]:
from pyspark.sql.types import *
import pyspark.sql.functions as f
import re
import time

In [2]:
df = spark.createDataFrame([
    ('id_cliente-1',  'cat-1, cat-2, cat-3'),
    ('id_cliente-2',  'cat-1, cat-4, cat-5'),
    ('id_cliente-3',  'cat-6, cat-7'),
    ('id_cliente-4',  'cat-1, cat-2, cat-7, cat-10'),
    ('id_cliente-5',  'cat-8, cat-10'),
    ('id_cliente-6',  'cat-1, cat-9, cat-10'),
    ('id_cliente-7',  'cat-1, cat-4, cat-5, cat-10'),
    ('id_cliente-8',  'cat-7, cat-9'),
    ('id_cliente-9',  'cat-1'),
    ('id_cliente-10', 'cat-1, cat-2, cat-3, cat-4, cat-5, cat-6, cat-7, cat-8, cat-10')
], ['id_cliente', 'categorias'])

In [3]:
df2 = spark.createDataFrame([
    ('id_cliente-1',  'cat-1, cat-2, cat-3, cat-15'),
    ('id_cliente-2',  'cat-1, cat-4, cat-5, cat-11, cat-14'),
    ('id_cliente-3',  'cat-4, cat-14, cat-15'),
    ('id_cliente-4',  'cat-1, cat-2, cat-7, cat-10'),
    ('id_cliente-5',  'cat-8, cat-10, cat-11'),
    ('id_cliente-6',  'cat-1, cat-9, cat-10, cat-11, cat-13'),
    ('id_cliente-7',  'cat-1, cat-4, cat-5, cat-10'),
    ('id_cliente-8',  'cat-7, cat-9, cat-12, cat-13, cat-14'),
    ('id_cliente-9',  'cat-2'),
    ('id_cliente-10', 'cat-1, cat-2, cat-3, cat-4, cat-5, cat-6, cat-7, cat-8, cat-10')
], ['id_cliente', 'categorias'])

In [4]:
df1_ = spark.createDataFrame([''], ['id_cliente', 'cat-1', 'cat-2', 'cat-3', 'cat-4', 'cat-5', 'cat-6', 'cat-7', 'cat-8', 'cat-9', 'cat-10'])

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-1163559722078741> in <module> () 
 ----> 1 df1_ = spark . createDataFrame ( [ '' ] , [ 'id_cliente' , 'cat-1' , 'cat-2' , 'cat-3' , 'cat-4' , 'cat-5' , 'cat-6' , 'cat-7' , 'cat-8' , 'cat-9' , 'cat-10' ] ) 

 /databricks/spark/python/pyspark/sql/session.py in createDataFrame (self, data, schema, samplingRatio, verifySchema) 
 808 rdd , schema = self . _createFromRDD ( data . map ( prepare ) , schema , samplingRatio ) 
 809 else : 
 --> 810 rdd , schema = self . _createFromLocal ( map ( prepare , data ) , schema ) 
 811 jrdd = self . _jvm . SerDeUtil . toJavaArray ( rdd . _to_java_object_rdd ( ) ) 
 812 jdf = self . _jsparkSession . applySchemaToPythonRDD ( jrdd . rdd ( ) , schema . json ( ) ) 

 /databricks/spark/python/pyspark/sql/session.py in _createFromLocal (self, data, schema) 
 440 write temp files . 
 441 """
 --> 442 data , schema = self . _wrap_data_schema ( data , schema ) 
 443 return self . _sc . parallelize ( data ) , schema 
 444 

 /databricks/spark/python/pyspark/sql/session.py in _wrap_data_schema (self, data, schema) 
 419 
 420 if schema is None or isinstance ( schema , ( list , tuple ) ) : 
 --> 421 struct = self . _inferSchemaFromList ( data , names = schema ) 
 422 converter = _create_converter ( struct ) 
 423 data = map ( converter , data ) 

 /databricks/spark/python/pyspark/sql/session.py in _inferSchemaFromList (self, data, names) 
 355 warnings.warn("inferring schema from dict is deprecated,"
 356 "please use pyspark.sql.Row instead")
 --> 357 schema = reduce ( _merge_type , ( _infer_schema ( row , names ) for row in data ) ) 
 358 if _has_nulltype ( schema ) : 
 359 raise ValueError ( "Some of types cannot be determined after inferring" ) 

 /databricks/spark/python/pyspark/sql/session.py in <genexpr> (.0) 
 355 warnings.warn("inferring schema from dict is deprecated,"
 356 "please use pyspark.sql.Row instead")
 --> 357 schema = reduce ( _merge_type , ( _infer_schema ( row , names ) for row in data ) ) 
 358 if _has_nulltype ( schema ) : 
 359 raise ValueError ( "Some of types cannot be determined after inferring" ) 

 /databricks/spark/python/pyspark/sql/types.py in _infer_schema (row, names) 
 1060 
 1061 else : 
 -> 1062 raise TypeError ( "Can not infer schema for type: %s" % type ( row ) ) 
 1063 
 1064 fields = [ StructField ( k , _infer_type ( v ) , True ) for k , v in items ] 

 TypeError : Can not infer schema for type: <class 'str'>

In [5]:
df3 = df.select(
        "id_cliente",
        f.split("categorias", ", ").alias("categorias"),
        f.posexplode(f.split("categorias", ", ")).alias("pos", "val")
    )\
    .drop("val").show()


+------------+--------------------+---+
 id_cliente| categorias|pos|
+------------+--------------------+---+
id_cliente-1|[cat-1, cat-2, ca...| 0|
id_cliente-1|[cat-1, cat-2, ca...| 1|
id_cliente-1|[cat-1, cat-2, ca...| 2|
id_cliente-2|[cat-1, cat-4, ca...| 0|
id_cliente-2|[cat-1, cat-4, ca...| 1|
id_cliente-2|[cat-1, cat-4, ca...| 2|
id_cliente-3| [cat-6, cat-7]| 0|
id_cliente-3| [cat-6, cat-7]| 1|
id_cliente-4|[cat-1, cat-2, ca...| 0|
id_cliente-4|[cat-1, cat-2, ca...| 1|
id_cliente-4|[cat-1, cat-2, ca...| 2|
id_cliente-4|[cat-1, cat-2, ca...| 3|
id_cliente-5| [cat-8, cat-10]| 0|
id_cliente-5| [cat-8, cat-10]| 1|
id_cliente-6|[cat-1, cat-9, ca...| 0|
id_cliente-6|[cat-1, cat-9, ca...| 1|
id_cliente-6|[cat-1, cat-9, ca...| 2|
id_cliente-7|[cat-1, cat-4, ca...| 0|
id_cliente-7|[cat-1, cat-4, ca...| 1|
id_cliente-7|[cat-1, cat-4, ca...| 2|
+------------+--------------------+---+
only showing top 20 rows

In [6]:
def pegaNumero(x):
  return(int(re.findall(r'\d+', x)[0]))

def transforma(x):
  return('client-uuid-'+str(x))

pegaNumeroUDF = udf(pegaNumero, IntegerType())
transformaUDF = udf(transforma, StringType())

In [7]:
x1 = df.select('id_cliente', explode(split("categorias",",")).alias("opa")).withColumn("opa", trim(col("opa")))
x1 = x1.withColumn('id', pegaNumeroUDF("id_cliente"))
x1 = x1.withColumn('opa_id', pegaNumeroUDF("opa"))
x1 = x1.groupBy('id_cliente','id').pivot('opa_id').count().na.fill(0).sort('id', ascending=True)
x1 = x1.withColumn('id', transformaUDF("id"))
replacements = {c:'cat-'+str(c) for c in x1.columns if c.isnumeric()}
x1 = x1.select([col(c).alias(replacements.get(c, c)) for c in x1.columns])
x1.drop('id_cliente').show()


+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+
 id|cat-1|cat-2|cat-3|cat-4|cat-5|cat-6|cat-7|cat-8|cat-9|cat-10|
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+
 client-uuid-1| 1| 1| 1| 0| 0| 0| 0| 0| 0| 0|
 client-uuid-2| 1| 0| 0| 1| 1| 0| 0| 0| 0| 0|
 client-uuid-3| 0| 0| 0| 0| 0| 1| 1| 0| 0| 0|
 client-uuid-4| 1| 1| 0| 0| 0| 0| 1| 0| 0| 1|
 client-uuid-5| 0| 0| 0| 0| 0| 0| 0| 1| 0| 1|
 client-uuid-6| 1| 0| 0| 0| 0| 0| 0| 0| 1| 1|
 client-uuid-7| 1| 0| 0| 1| 1| 0| 0| 0| 0| 1|
 client-uuid-8| 0| 0| 0| 0| 0| 0| 1| 0| 1| 0|
 client-uuid-9| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0|
client-uuid-10| 1| 1| 1| 1| 1| 1| 1| 1| 0| 1|
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+